# Comparing Several Discrete Choice Models

- - -

W. Ross Morrow ([wrossmorrow@stanford.edu](mailto:wrossmorrow@stanford.edu), [wrossmorrow.com](https://wrossmorrow.com))

Research Analytics Consultant, Stanford GSB

February 5th, 2019

- - -

# Introduction 

- - - - 

In this notebook we'll compare several Discrete Choice Models in a simple situation: single-feature binary choice data. Specifically, the data will be composed of a number of observations, $N$, a number of individuals, $I$, an individual index per observations $i_n$, and a "yes/no" dummy $y_n \in \{\pm 1\}$ standing in for random (but structured) choices. We can draw those choices with a variety of data generating processes, as discussed below. But the basic setup is that we see some number of repeated trials for different individuals in which they answer "yes" or "no" to some question, and our broad goal is to model the frequency with which respondents say "yes" or "no". 

First, lets do our basic imports and function definitions. We'll discuss the models for data $(N,I,\mathbf{i},\mathbf{y})$ below. 

In [254]:
import os
import argparse
import time
from multiprocessing import Pool , Queue , Process
from multiprocessing.pool import ThreadPool

import string
import random

import numpy as np
from numpy.random import rand , randn , randint , choice

import cvxpy as cp
import ecos

from scipy.sparse import csc_matrix , csr_matrix , coo_matrix , issparse
from scipy.optimize import minimize , LinearConstraint , BFGS
from scipy.stats import norm as gaussian , normaltest

import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
%matplotlib notebook
        

These are some wrappers around random number generation that are useful here. 

In [255]:

def randi(A,N) : return randint(0,high=A,size=N) 

def rands(N) : return np.sign( 2.0*rand(N) - 1.0 )

def randc(C,N,p) : return choice( C , size=N , p=p )

def randp(A,N) : 
    L , R = randi(A,N) , randi(A-1,N)
    R[ np.where( R >= L )[0] ] += 1
    return L , R

def random_string( l ) : 
    return ''.join( random.choice(string.ascii_letters) for m in range(l) )


This is a nice simple way to plot sparsity patterns for not-too-large sparse matrices. 

In [256]:

def spy( A , b=None ) : 
    plt.figure( figsize=(10,3) )
    plt.xticks([]) ; plt.yticks([])
    if issparse(A) : A = A.toarray()
    if b is None : 
        plt.imshow( A != 0 , cmap='Greys' , interpolation='nearest', aspect='auto' )
    else : 
        X = np.zeros( ( A.shape[0] , A.shape[1]+4 ) )
        X[:,:A.shape[1]] = A
        X[:,A.shape[1]+3] = b
        plt.imshow( X != 0 , cmap='Greys' , interpolation='nearest', aspect='auto' )
        

# Models

- - -

Below we review Logit, Latent Class Logit, Random Coefficient Logit, and `idLogit` models and provide code for this specific (and rather simple) situation. 

First, however, we define a general class `FittableModel` each model-specific class will be derived from. This class has a timeout-enabled fitting method as well as derivative checking routines for gradients and Hessian-vector products. The timeout-enabled fitting method is optional, but is helpful given that the models we try to fit can have dramatically different fit times. 

In [257]:
class FittableModel( object ) : 
    
    def __init__( self ) : 
        pass
    
    def fit( self , p0=None , maxtime=None ) : 
        
        if p0 is None : 
            try : 
                p0 = self.draw_initial_condition(  )
            except AttributeError as e : 
                p0 = randn( self.Nvars )
        else : 
            p0 = p0.flatten()
            
        start = time.time()
        
        if ( maxtime is None ) or ( maxtime <= 0.0 ) : 
            
            self.soln = self.wrapped_solve( p0=p0 )
            self.soln['timeout'] = False
            
        else : 
            
            q = Queue()
            p = Process( target=self.wrapped_solve , kwargs={ 'p0' : p0 , 'queue' : q } )
            
            p.start()
            while time.time() - start < maxtime :
                p.join( timeout=1 )
                if not p.is_alive() : break
            if p.is_alive():
                p.terminate()
                self.soln = { 'timeout' : True }
            else : 
                self.soln = q.get()
                self.soln['timeout'] = False
                
        self.soln['solvertime'] = time.time() - start
        self.soln['x0'] = p0
        
        return self
    
    def wrapped_solve( self , p0=None , queue=None ) :
        try : 
            self.soln = self.solve( p0=p0 )
            self.soln['error'] = False
        except Exception as e : 
            print( "caught solve exception: " , e )
            self.soln = { 'error' : True , 'message' : e }
        if queue is not None : queue.put( self.soln )
        return self.soln
    
    def grad_check( self , p=None , verbose=True ) : 

        """
        gradient check of an object that has certain attributes (Nvars, obj, and grad)

        """

        if p is None : p = rand( self.Nvars )

        f0 , g0 = self.obj( p ) , self.grad( p ) # original objective and gradient

        Hs = 10.0**np.arange( -10 , 0 , 1 )[::-1] # perturbation sizes
        pP = p.copy() # copy for perturbed betas

        df = np.zeros( self.Nvars ) # space for dinite differences
        ds = np.zeros( Hs.size ) # differences between gradient and finite differences
        
        # iterations
        for h in range( Hs.size ) : 
            H = Hs[h] # actual perturbation
            for k in range( self.Nvars ) : 
                pP[k] += H # perturb on coordinate k
                fk = self.obj( pP ) # evalute objective at perturbed argument
                df[k] = ( fk - f0 ) / H # compute finite difference
                pP[k] -= H # perturb on coordinate k
            ds[h] = np.max( np.abs( g0 - df ) ) # compute difference

        if( verbose ) : 
            for h in range( Hs.size ) : 
                print( "%0.16f , %0.16f" % ( Hs[h] , ds[h] ) )

        return Hs , ds

    def hessp_check( self , p=None , v=None , verbose=True ) :  

        """
        hessian product check of an object that has certain attributes (Nvars, grad, and hessp)

        """

        if p is None : p = rand( self.Nvars )
        if v is None : v = rand( self.Nvars )

        g0 , h0 = self.grad( p ) , self.hessp( p , v ) # original gradient

        Hs = 10.0**np.arange( -10 , 1 , 1 )[::-1] # perturbation sizes
        pP = p.copy() # copy for perturbed betas

        ds = np.zeros( Hs.size ) # differences between gradient and finite differences

        # iterations
        for h in range( Hs.size ) : 
            H = Hs[h] # actual perturbation
            pP = p + H * v # perturb by H in direction v
            gP = self.grad( pP ) # evalute objective gradient at perturbed argument
            df = ( gP - g0 ) / H
            ds[h] = np.max( np.abs( df - h0 ) ) # compute difference

        if( verbose ) : 
            for h in range( Hs.size ) : 
                print( "%0.16f , %0.16f" % ( Hs[h] , ds[h] ) )

        return Hs , ds
    

Note that we have a specific way of testing derivative accuracy: Compare finite differences (for gradients or for hessian-vector products) for a _decreasing sequence_ of perturbation sizes. Regardless of the function evaluation accuracy, we should see a "V" shape in the associated relative error magnitude between computed derivatives and finite differences. The finite difference approximation should be innaccurate for "large" perturbations but get more accurate as the perturbation decreases, that is until floating point errors start to accumulate and reduce the approximation accuracy. 

This is obviously more computationally intensive that comparing derivatives at a single perturbation size and for large enough problems can be quite burdensome. However single-point approximations give no indication of what a "reasonable" error should be, and thus are somewhat useless when we don't know how much error exists in our function and derivative evaluations to begin with. 

## Logit

- - -

The Logit is the simplest model, with "yes" probability
$$
    \frac{e^\beta}{1+e^\beta}
$$
and MLE problem
$$
\begin{aligned}
    \min &\quad \frac{1}{N} \sum_{n=1}^N \log( 1 + e^{-y_n\beta} ) \\
    \text{w.r.t.} &\quad \beta \in \mathbb{R}
\end{aligned}
$$
The derivative of the objective is easily derived as
$$
    D^\beta = \frac{1}{N} \sum_{n=1}^N (-y_n)\left( \frac{ e^{-y_n\beta} }{ 1 + e^{-y_n\beta} } \right)
$$
Moreover the Hessian is even easily derived as
$$
    \frac{1}{N} \sum_{n=1}^N \left( \frac{ e^{-y_n\beta} }{ 1 + e^{-y_n\beta} } \right)\left( 1 - \frac{ e^{-y_n\beta} }{ 1 + e^{-y_n\beta} } \right)
$$
The following class, derived from `FittableModel`, implements the Logit. 

In [319]:
class Logit( FittableModel ) : 
    
    type = "Logit"
    
    def __init__( self , N , y ) : 
        self.Nvars , self.N , self.y , self.ny = 1 , N , y , - y

    def obj( self , p ) :
        return np.sum( np.log1p( np.exp( self.ny * p ) ) ) / self.N

    def grad( self , p ) :  
        eU = np.exp( self.ny * p ) ; PL = np.divide( eU , 1.0 + eU )
        return np.array( [ np.sum( PL * self.ny ) / self.N ] )
    
    def hess( self , p ) :  
        eU = np.exp( self.ny * p ) ; PL = eU / ( 1.0 + eU )
        return np.array( [ np.sum( PL * ( 1.0 - PL ) ) / self.N ] )
    
    def solve( self , p0=None ) : 
        self.soln = minimize( self.obj , p0 , jac=self.grad , \
                              hess=self.hess , method='trust-constr' )
        return self.soln
    
    def printx( self ) : 
        if self.soln is None : return "(no solution)"
        return "%0.2f" % self.soln['x'][0]
    
    def getx( self ) : 
        if self.soln is None : return None
        else : return self.soln['x'][0]
    
    def kld( self , PT ) : 
        if self.soln['x'][0] > 0 : 
            P = 1.0 / ( 1.0 + np.exp( -self.soln['x'][0] ) )
        else : 
            P = np.exp( self.soln['x'][0] ) ; P = P / ( 1.0 + P )
        return PT * np.log( PT/P ) + (1.0-PT) * np.log( (1.0-PT)/(1.0-P) )
        

## Latent Class Logit

- - -

Here we presume that each individual is "drawn" from one of $C$ classes each with its own coefficient. Our job is to estimate the class coefficients and the mass function for the classes. The simplest version of this problem is: 
$$
\begin{aligned}
\\
    \min &\quad - \frac{1}{N} \sum_{i=1}^I \log \left( \sum_{c=1}^C \rho_c e^{L_i(\beta_c)} \right)    
        \quad\quad\text{where}\quad 
        L_i(\theta) = - \sum_{ n \in \mathcal{O}_i } \log \Big( 1 + e^{-y_n\theta} \Big) 
        \\
    \text{w.r.t.} &\quad 0 \leq \rho_c \leq 1 \;\; , \;\; \beta_c \in \mathbb{R} 
                        \;\; \text{for all} \;\; c = 1,\dotsc,C \\
    \text{s.to} &\quad \sum_{c=1}^C \rho_c = 1 \\
   \\
\end{aligned}
$$
The derivatives are
$$
\begin{aligned}
    D_c^\rho 
        &= - \frac{1}{N} \sum_{i=1}^I \frac{ e^{L_i(\beta_c)} }{ \sum_{d=1}^C \rho_d e^{L_i(\beta_d)} }
    \\
    D_c^\beta 
        &= - \frac{1}{N} \sum_{i=1}^I \frac{ \rho_c e^{L_i(\beta_c)} L_i^\prime(\beta_c) }{ \sum_{d=1}^C \rho_d e^{L_i(\beta_d)} }
    \quad\quad\text{where}\quad 
    L_i^\prime(\theta) 
        = - \sum_{ n \in \mathcal{O}_i } (-y_n)\frac{ e^{-y_n\theta} }{ 1 + e^{-y_n\theta} }
        = \sum_{ n \in \mathcal{O}_i } y_n\frac{ e^{-y_n\theta} }{ 1 + e^{-y_n\theta} }
\end{aligned}
$$

### Stabilization

We have to consider an important stabilization technique. If $\max_c L_i( \beta_c ) \ll 0$, then $e^{ L_i( \beta_c ) } \approx 0$ and 
$$
    \log \left( \sum_{c=1}^C \rho_c e^{ L_i( \beta_c ) } \right )
$$ 
will not be computable. Specifically, if 
$$
\begin{aligned}
    \\
    \max_c L_i( \beta_c ) \leq -745.15
    \quad\quad\text{then}\quad\quad
    \texttt{float}( e^{ L_i( \beta_c ) } ) = 0
    \\
    \\
\end{aligned}
$$
where $\texttt{float}$ is the floating point representation of the exponential (in double precision). This has to be handled carefully if we want to allow for sufficient exploration of $\beta_1,\dotsc,\beta_C$ for arbitrary data. 

In particular, note that 
$$
    L_i(\theta) = - |\{n:y_n=+1\}| \log\left( 1 + e^{-\theta} \right) - |\{n:y_n=-1\}| \log\left( 1 + e^{\theta} \right)
$$
This is, in fact, a formula we could use to dramatically improve the scalability of the fitting methods we explore, but at the cost of _complete_ loss of generality. What is worth noting here is that as the number of observations increase, the size of both sets goes to infinity; moreover the $\log$ terms are both positive, having arguments larger than one. Thus, for any finite $\theta$, there is _some_ number of observations such that $L_i$ is too negative to effectively compute with. It is probably even possible to analyze this situation further, but knowing exactly _when_ we get into trouble isn't helpful. 

A simple trick sidesteps this potential problem: Let 
$$
L_i^*(\boldsymbol{\beta}) = \max_c L_i( \beta_c )
$$
and then
$$
   -\frac{1}{N} \sum_{i=1}^I \log \left( \sum_{c=1}^C \rho_c e^{ L_i( \beta_c ) } \right )
        = -\frac{1}{N} \sum_{i=1}^I L_i^*(\boldsymbol{\beta}) -\frac{1}{N} \sum_{i=1}^I  \log \left( \sum_{c=1}^C \rho_c e^{ L_i( \beta_c ) - L_i^*(\boldsymbol{\beta}) } \right )
$$ 
Note also that the derivatives can be evaluated with $L_i( \beta_c ) - L_i^*(\boldsymbol{\beta})$: 
$$
\begin{aligned}
    D_c^\rho 
        &= - \frac{1}{N} \sum_{i=1}^I \frac{ e^{L_i(\beta_c)} }{ \sum_{d=1}^C \rho_d e^{L_i(\beta_d)} }
        = - \frac{1}{N} \sum_{i=1}^I \frac{ e^{L_i^*(\boldsymbol{\beta})} e^{L_i(\beta_c)-L_i^*(\boldsymbol{\beta})} }{ e^{L_i^*(\boldsymbol{\beta})} \sum_{d=1}^C \rho_d e^{L_i(\beta_d)-L_i^*(\boldsymbol{\beta})} }
        = - \frac{1}{N} \sum_{i=1}^I \frac{ e^{L_i(\beta_c)-L_i^*(\boldsymbol{\beta})} }{\sum_{d=1}^C \rho_d e^{L_i(\beta_d)-L_i^*(\boldsymbol{\beta})} }
    \\
    D_c^\beta 
        &= - \frac{1}{N} \sum_{i=1}^I \frac{ \rho_c e^{L_i(\beta_c)} L_i^\prime(\beta_c) }{ \sum_{d=1}^C \rho_d e^{L_i(\beta_d)} }
        = - \frac{1}{N} \sum_{i=1}^I \frac{ e^{L_i^*(\boldsymbol{\beta})} \rho_c e^{L_i(\beta_c)-L_i^*(\boldsymbol{\beta})} L_i^\prime(\beta_c) }{ e^{L_i^*(\boldsymbol{\beta})} \sum_{d=1}^C \rho_d e^{L_i(\beta_d)-L_i^*(\boldsymbol{\beta})} }
        = - \frac{1}{N} \sum_{i=1}^I \frac{ \rho_c e^{L_i(\beta_c)-L_i^*(\boldsymbol{\beta})} L_i^\prime(\beta_c) }{ \sum_{d=1}^C \rho_d e^{L_i(\beta_d)-L_i^*(\boldsymbol{\beta})} }
    \\
\end{aligned}
$$

In [320]:
class LatentClassLogit( FittableModel ) : 
    
    type = "Latent Class Logit"
    
    def __init__( self , N , I , i , y , C , ordered=True ) : 
        
        if C <= 1 : 
            raise ArgumentError( "Trivial number of classes: should be at least two." )
        
        self.N , self.I , self.i , self.y , self.C = N , I , i , -y , C
        self.Nvars , self.Ncons = 2*C , 2*C if ordered else C+1
        
        # this matrix "assigns" observations to individuals, facilitating 
        # sums over observations within individuals
        self.reducer = csr_matrix( (-np.ones(N),(i,np.arange(N))) , shape=(I,N) )
        self.yeducer = csr_matrix( (y,(i,np.arange(N))) , shape=(I,N) )
        
        # constraints: 
        #  
        #     p[0] , p[1] , ... , p[C] >= 0.0
        #     p[0] + p[1] + ... + p[C] = 1.0
        #     p[0] >= p[1] >= ... >= p[C] if ordered
        # 
        
        lo = np.zeros( self.Ncons )
        up = np.inf * np.ones( self.Ncons )
        lo[C] , up[C] = 1.0 , 1.0
        Cmtrx = np.zeros( ( self.Ncons , 2*C ) )
        for c in range(self.C) : 
            Cmtrx[c,c] = 1.0 # p[c] >= 0
            Cmtrx[C,c] = 1.0 # sum_c p[c] = 1
        if ordered : 
            for c in range(1,self.C) : 
                Cmtrx[C+c,c-1] = 1.0
                Cmtrx[C+c,c] = - 1.0
            
        self.cons = LinearConstraint( Cmtrx , lo , up )
        
        # workspace
        self.yp = np.zeros((self.N,self.C),dtype=np.float)
        self.eU = np.zeros((self.N,self.C),dtype=np.float)
        self.ll = np.zeros((self.N,self.C),dtype=np.float)
        self.Li = np.zeros((self.I,self.C),dtype=np.float)
        self.EL = np.zeros((self.I,self.C),dtype=np.float)
        self.PL = np.zeros((self.N,self.C),dtype=np.float)
        self.LP = np.zeros((self.I,self.C),dtype=np.float)
        self.j  = np.zeros((self.I,),dtype=np.float)
        self.LM = np.zeros((self.I,),dtype=np.float)
        
    def basics( self , p ) : 
        np.outer( self.y , p[self.C:] , out=self.yp )
        np.exp( self.yp , out=self.eU )
        np.log1p( self.eU , out=self.ll )
        self.Li = self.reducer @ self.ll
        np.max( self.Li , axis=1 , out=self.LM )
        self.Li = self.Li - np.tile( self.LM.reshape((self.I,1)) , (1,self.C) )
        np.exp( self.Li , out=self.EL )
        self.j = self.EL @ p[:self.C]
        
    def obj( self , p ) :
        self.basics( p )
        np.log( self.j , out=self.j )
        return - np.sum( self.LM + self.j ) / self.N

    def grad( self , p ) :  
        self.basics( p ) 
        np.divide( self.eU , 1.0 + self.eU , out=self.PL )
        self.LP = self.yeducer @ self.PL
        self.j  = 1.0 / self.j
        
        # gradient terms
        g = np.zeros( self.Nvars )
        g[:self.C] = - self.EL.T @ self.j / self.N
        g[self.C:] = - p[:self.C] * ( ( self.EL * self.LP ).T @ self.j ) / self.N
        
        return g
    
    def draw_initial_condition( self ) : 
        p0 = randn( self.Nvars ) 
        p0[:self.C] = np.abs( p0[:self.C] )
        p0[:self.C] = p0[:self.C] / p0[:self.C].sum()
        return p0
    
    def solve( self , p0=None ) : 
        self.soln = minimize( self.obj , p0 , jac=self.grad , hess=BFGS() , \
                            method='trust-constr' , constraints=self.cons , \
                           options={ 'maxiter' : 100000 } )
        return self.soln
    
    def printx( self ) : 
        if self.soln is None : return "(no solution)"
        s = " ) , ( ".join( [ "%0.2f , %0.2f" % ( self.soln['x'][c+self.C] , self.soln['x'][c] ) \
                                    for c in range(self.C) ] )
        return "( %s )" % s
    
    def getx( self ) : 
        if self.soln is None : return None
        else : 
            return self.soln['x'][self.C:]
    
    def kld( self , PT ) : 
        PL = np.exp( self.soln['x'][self.C:] ) ; PL = PL / ( 1.0 + PL )
        P  = np.sum( PL * self.soln['x'][:self.C] )
        return PT * np.log( PT/P ) + (1.0-PT) * np.log( (1.0-PT)/(1.0-P) )


## "Classification" Latent Class Logit

- - -

Here we "classify" individuals to classes, instead of presume each individual was drawn from a class at random. For this we introduce individual-specific class membership probabilities and solve
$$
\begin{aligned}
    \min &\quad - \frac{1}{N} \sum_{i=1}^I \log \left( \sum_{c=1}^C \rho_{i,c} e^{L_i(\beta_c)} \right) \\
    \text{w.r.t.} &\quad 0 \leq \rho_{i,c} \leq 1 \;\; , \;\; \beta_c \in \mathbb{R} 
                        \;\; \text{for all} \;\; c = 1,\dotsc,C \\
    \text{s.to} &\quad \sum_{c=1}^C \rho_{i,c} = 1 \text{ for all } i \\
\end{aligned}
$$
The derivatives here are simpler, if more numerous: 
$$
\begin{aligned}
    D_{i,c}^\rho 
        &= - \frac{1}{N} \frac{ e^{L_i(\beta_c)} }{ \sum_{d=1}^C \rho_{i,d} e^{L_i(\beta_d)} }
        \\
    D_{c}^\beta 
        &= - \frac{1}{N} \sum_{i=1}^I \frac{ \rho_{i,c} e^{L_i(\beta_c)} }{ \sum_{d=1}^C \rho_{i,d} e^{L_i(\beta_d)} }
            L_i^\prime(\beta_c)
        \\
\end{aligned}
$$
This problem is also not always well-determined, as the large number of variables ($(I+1)C$) can exceed the number of observations ($N$). 

We stabilize in the same way as described above. 

In [321]:
class ClassClassLogit( FittableModel ) : 
    
    type = "Class Class Logit"
    
    def __init__( self , N , I , i , y , C , ordered=True ) : 
        
        if C <= 1 : 
            raise ArgumentError( "Trivial number of classes: should be at least two." )
        
        self.N , self.I , self.i , self.y , self.C = N , I , i , -y , C
        self.Nvars , self.IC , self.Ncons = (I+1)*C , I*C , I*(C+1)
        
        if self.N - self.Nvars <= 0 : 
            raise ArgumentError( "Not enough observations for the number of individuals and classes." )
        
        # this matrix "assigns" observations to individuals, facilitating 
        # sums over observations within individuals
        self.reducer = csr_matrix( (-np.ones(N),(i,np.arange(N))) , shape=(I,N) )
        self.yeducer = csr_matrix( (y,(i,np.arange(N))) , shape=(I,N) )
        
        # constraints: 
        #  
        #     p[0] , p[1] , ... , p[IC-1] >= 0.0                          IC equations
        #     p[i,0] + p[i,1] + ... + p[i,C-1] = 1.0 for all I            I equations
        #     p[i,0] - p[i,1] , ... p[i,C] - p[i,C-1] >= 0 if ordered ?   I(C-1) equations
        # 
        
        lo , up = np.zeros( self.Ncons ) , np.inf * np.ones( self.Ncons )
        lo[I*C:I*(C+1)] , up[I*C:I*(C+1)] = 1.0 , 1.0
        
        Cnnzs = 2*I*C
        Crows , Ccols = np.zeros( Cnnzs , dtype=np.int ) , np.zeros( Cnnzs , dtype=np.int )
        Cdata = np.ones( Cnnzs )
        
        Crows[:I*C] , Ccols[:I*C] = np.arange(I*C) , np.arange(I*C)
        Crows[I*C:] , Ccols[I*C:] = I*C + np.repeat( np.arange(I) , C ) , np.arange(I*C)
        
        Cmtrx = csr_matrix( (Cdata,(Crows,Ccols)) , shape=( self.Ncons , self.Nvars ) )
            
        self.cons = LinearConstraint( Cmtrx , lo , up )
        
        # workspace
        self.yp = np.zeros((self.N,self.C),dtype=np.float)
        self.eU = np.zeros((self.N,self.C),dtype=np.float)
        self.ll = np.zeros((self.N,self.C),dtype=np.float)
        self.Li = np.zeros((self.I,self.C),dtype=np.float)
        self.EL = np.zeros((self.I,self.C),dtype=np.float)
        self.TL = np.zeros((self.I,self.C),dtype=np.float)
        self.PL = np.zeros((self.N,self.C),dtype=np.float)
        self.LP = np.zeros((self.I,self.C),dtype=np.float)
        self.j  = np.zeros((self.I,),dtype=np.float)
        self.LM = np.zeros((self.I,),dtype=np.float)

    def basics( self , p ) : 
        np.outer( self.y , p[self.IC:] , out=self.yp )
        np.exp( self.yp , self.eU )
        np.log1p( self.eU , out=self.ll )
        self.Li = self.reducer @ self.ll
        np.max( self.Li , axis=1 , out=self.LM )
        self.Li = self.Li - np.tile( self.LM.reshape((self.I,1)) , (1,self.C) )
        np.exp( self.Li , out=self.EL )
        np.multiply( self.EL , p[:self.IC].reshape((self.I,self.C)) , out=self.TL )
        self.j = np.sum( self.TL , axis=1 )
        
    def obj( self , p ) :
        self.basics( p )
        np.log( self.j , out=self.j )
        return - np.sum( self.LM + self.j ) / self.N

    def grad( self , p ) :  
        
        self.basics( p )
        
        np.divide( self.eU , 1.0 + self.eU , out=self.PL )
        self.LP = self.yeducer @ self.PL
        self.j = 1.0 / self.j
        
        # gradient terms
        g = np.zeros( self.Nvars )
        g[:self.IC] = - self.EL.flatten() * np.repeat( self.j , self.C ) / self.N
        g[self.IC:] = - ( ( self.TL * self.LP ).T @ self.j ) / self.N
        
        return g
    
    def draw_initial_condition( self ) : 
        p0 = randn(self.Nvars)
        p0[:self.IC] = np.abs( p0[:self.IC] )
        for i in range( self.I ) : 
            p0[i*self.C:i*self.C+1] = p0[i*self.C:i*self.C+1] / p0[i*self.C:i*self.C+1].sum()
        return p0
        
    def solve( self , p0=None ) : 
        self.soln = minimize( self.obj , p0 , jac=self.grad , hess=BFGS() , \
                            method='trust-constr' , constraints=self.cons , \
                           options={ 'maxiter' : 100000 } )
        return self.soln
    
    def printx( self ) : 
        if self.soln is None : return "(no solution)"
        rho = self.soln['x'][:self.IC].reshape((self.I,self.C)).mean( axis=0 )
        s = " ) , ( ".join( [ "%0.2f , %0.2f" % ( self.soln['x'][self.IC+c] , rho[c] ) for c in range(self.C) ] )
        return "( %s )" % s
    
    def getx( self ) : 
        if self.soln is None : return None
        else : 
            w = self.soln['x'][:self.IC].reshape( (self.I,self.C) ).mean( axis=0 ) # average class likelihoods
            return self.soln['x'][self.IC:]
    
    def kld( self , PT ) : 
        PL = np.exp( self.soln['x'][self.IC:] ) ; PL = PL / ( 1.0 + PL )
        w  = self.soln['x'][:self.IC].reshape( (self.I,self.C) ).mean( axis=0 ) # average class likelihoods
        P  = np.sum( PL * w )
        return PT * np.log( PT/P ) + (1.0-PT) * np.log( (1.0-PT)/(1.0-P) )
    

## (Gaussian) Random Coefficients

- - -

The random (gaussian) coefficient Logit for this situation is
$$
\begin{aligned}
    \min &\quad -\frac{1}{N} \sum_{i=1}^I \log \left( 
                        \int e^{ L_i( \mu + \sigma v ) } \phi(v) dv
                    \right) \\
    \text{w.r.t.} &\quad \mu \in \mathbb{R} \; , \; \sigma \geq 0
\end{aligned}
$$
where $L_i$ has the same definition as above. Broadly speaking, the derivatives are
$$
\begin{aligned}
    D^\mu 
        &= -\frac{1}{N} \sum_{i=1}^I 
                            \frac{ \int e^{ L_i( \mu + \sigma v ) } L_i^\prime( \mu + \sigma v ) \phi(v) dv }
                                { \int e^{ L_i( \mu + \sigma v ) } \phi(v) dv } 
                    \\
    D^\sigma
        &= -\frac{1}{N} \sum_{i=1}^I 
                            \frac{ \int e^{ L_i( \mu + \sigma v ) } L_i^\prime( \mu + \sigma v ) v \phi(v) dv }
                                { \int e^{ L_i( \mu + \sigma v ) } \phi(v) dv }
                    \\
\end{aligned}
$$
we can approximate these directly, or approximate them by differentiating our approximation to the actual integral. 

Here we approximate the integrals with a sequence of $S$ "standardized" samples $v_s$ and associated weights $w_s$ that we hold fix over all individuals: 
$$
    \int e^{ L_i( \mu + \sigma v ) } \phi(v) dv
        \approx \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) }. 
$$
This makes the negative log likelihood approximation 
$$
    -\frac{1}{N} \sum_{i=1}^I \log\left( \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) } \right) .
$$
The associated (identically sampled and weighted) derivative approximations are
$$
\begin{aligned}
    D^\mu 
        &= -\frac{1}{N} \sum_{i=1}^I 
                            \frac{ \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) } L_i^\prime( \mu + \sigma v_s ) }
                                { \sum_{s=1}^S  w_s e^{ L_i( \mu + \sigma v_s ) }  } 
                    \\
    D^\sigma
        &= -\frac{1}{N} \sum_{i=1}^I 
                            \frac{ \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) } L_i^\prime( \mu + \sigma v_s ) v_s}
                                { \sum_{s=1}^S w_s  e^{ L_i( \mu + \sigma v_s ) } } 
                    \\
\end{aligned}
$$
It is important that we either hold such approximations fixed over the course of an estimation attempt or ensure that the approximation is so good that changing approximation error does not interfere with optimizer progress. 

Given $\mathbf{v}$ and $\mathbf{w}$, we can compute the negative log likelihood $f$ and gradient $\mathbf{g}$ as
$$
\begin{aligned}
    & \\
    &\quad \boldsymbol{\theta} \leftarrow \mu + \sigma \mathbf{v} \in \mathbb{R}^{S} \\
    &\quad \boldsymbol{\eta} \leftarrow \exp\{ - \mathbf{y} \boldsymbol{\theta}^\top \} \in \mathbb{R}^{N \times S} \\
    &\quad \boldsymbol{\ell} \leftarrow \log( 1 + \boldsymbol{\eta} ) \in \mathbb{R}^{N \times S} \\
    &\quad \mathbf{L} \leftarrow - \mathbf{R}\boldsymbol{\ell} \in \mathbb{R}^{I \times S} \\
    &\quad \mathbf{P} \leftarrow \boldsymbol{\eta} \; / \; (1+\boldsymbol{\eta}) \in \mathbb{R}^{N \times S} \\
    &\quad \mathbf{L}^\prime \leftarrow \mathbf{R}(\mathrm{diag}(\mathbf{y})\mathbf{P}) \in \mathbb{I \times S} \\
    &\quad \mathbf{E}_1 \leftarrow \exp\{ \mathbf{L} \} \in \mathbb{R}^{I \times S} \\
    &\quad \mathbf{E}_2 \leftarrow \mathbf{E}_1 * \mathbf{L}^\prime \in \mathbb{R}^{I \times S} \\
    &\quad\begin{aligned}
        &\texttt{for } i=1,\dotsc,I \\
        &\quad\quad \mathbf{E}_3[i,:] \leftarrow \mathbf{E}_2[i,:] * \mathbf{v}[:] \\
     \end{aligned} \quad\quad\text{or}\quad\quad
     \begin{aligned}
        &\texttt{for } s=1,\dotsc,S \\
        &\quad\quad \mathbf{E}_3[:,s] \leftarrow \mathbf{v}[s]\;\mathbf{E}_2[:,s] \\
     \end{aligned}
    \\
    &\quad\begin{aligned}
        &\texttt{for } k=1,2,3 \\
        &\quad\quad \mathbf{j}_k \leftarrow \mathbf{E}_k \mathbf{w} \in \mathbb{R}^{I} \\
     \end{aligned}
     \\
    &\quad f \leftarrow - \; \texttt{sum}( \; \log( \; \mathbf{j}_1 \; ) \; ) \; / \; N \\
    &\quad \mathbf{g} \leftarrow 
        - \frac{1}{N} \begin{pmatrix} 
            \texttt{sum}( \; \mathbf{j}_2 \; / \; \mathbf{j}_1 \; ) \\
            \texttt{sum}( \; \mathbf{j}_3 \; / \; \mathbf{j}_1 \; ) \\
        \end{pmatrix}
     \\
     & \\
\end{aligned}
$$
After defining a class for this approach we review two ways of computing sample points and weights: sample average approximations and Gauss-Legendre quadrature. 

### Stabilization

We have to again consider our stabilization. If $\max_s L_i( \mu + \sigma v_s ) \ll 0$, then $e^{ L_i( \mu + \sigma v_s ) } \approx 0$; if the latter holds, the resulting $\log$ will fail. Specifically, if 
$$
\begin{aligned}
    \\
    \max_s L_i( \mu + \sigma v_s ) \leq -745.15
    \quad\quad\text{then}\quad\quad
    \texttt{float}( e^{ L_i( \mu + \sigma v_s ) } ) = 0
    \\
    \\
\end{aligned}
$$
where $\texttt{float}$ is the floating point representation of the exponential (in double precision). This has to be handled carefully if we want to allow for sufficient exploration of $\mu,\sigma$ for arbitrary data. 

As above, let 
$$
L_i^*(\mu,\sigma) = \max_s L_i( \mu + \sigma v_s )
$$
and note that
$$
\begin{aligned}
    -\frac{1}{N} \sum_{i=1}^I \log\left( \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) } \right) 
        &= -\frac{1}{N} \sum_{i=1}^I \log\left( e^{L_i^*(\mu,\sigma)} \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) - L_i^*(\mu,\sigma) } \right) \\
        &= -\frac{1}{N} \sum_{i=1}^I L_i^*(\mu,\sigma) - \frac{1}{N} \sum_{i=1}^I \log\left( \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) - L_i^*(\mu,\sigma) } \right) \\
\end{aligned}
$$
Here at least one of the exponents, for any $i$, is zero by definition and thus its corresponding exponentiated value is one. Thus the sum is at least as large as the smallest weight, which is nonzero if we carefully construct the weights. 

If we want, we can absorb the weights too: 
$$
\begin{aligned}
    -\frac{1}{N} \sum_{i=1}^I \log\left( \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) } \right) 
        &= -\frac{1}{N} \sum_{i=1}^I \log\left( \sum_{s=1}^S e^{ W_{i,s}( \mu + \sigma v_s ) } \right) 
            &&\quad W_{i,s}(\theta) = L_i(\theta) + \log w_s\\
        &= -\frac{1}{N} \sum_{i=1}^I \log\left( e^{W_i^*(\mu,\sigma)} \sum_{s=1}^S e^{ W_{i,s}( \mu + \sigma v_s ) - W_i^*(\mu,\sigma) } \right) 
            &&\quad W_i^*(\mu,\sigma) = \max_s W_{i,s}(\mu+\sigma v_s) \\
        &= -\frac{1}{N} \sum_{i=1}^I W_i^*(\mu,\sigma) - \frac{1}{N} \sum_{i=1}^I \log\left( \sum_{s=1}^S e^{ W_{i,s}( \mu + \sigma v_s ) - W_i^*(\mu,\sigma) } \right) \\
\end{aligned}
$$
This form would ensure that the $\log$ is of a term that is always greater than one. 

Presuming the first form, the associated derivative approximations can be evaluated the same way with shifted values for $L_i(\mu+\sigma v_s)$: 
$$
\begin{aligned}
    D^\mu 
        &= -\frac{1}{N} \sum_{i=1}^I 
                            \frac{ \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) } L_i^\prime( \mu + \sigma v_s ) }
                                { \sum_{s=1}^S  w_s e^{ L_i( \mu + \sigma v_s ) }  } 
                    \\
        &= -\frac{1}{N} \sum_{i=1}^I 
                            \frac{ e^{L_i^*(\mu,\sigma)} \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) - L_i^*(\mu,\sigma) } L_i^\prime( \mu + \sigma v_s ) }
                                { e^{L_i^*(\mu,\sigma)} \sum_{s=1}^S  w_s e^{ L_i( \mu + \sigma v_s ) - L_i^*(\mu,\sigma) }  } 
                    \\
        &= -\frac{1}{N} \sum_{i=1}^I 
                            \frac{ \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) - L_i^*(\mu,\sigma) } L_i^\prime( \mu + \sigma v_s ) }
                                { \sum_{s=1}^S  w_s e^{ L_i( \mu + \sigma v_s ) - L_i^*(\mu,\sigma) }  } 
                    \\
    D^\sigma
        &= -\frac{1}{N} \sum_{i=1}^I 
                            \frac{ \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) } L_i^\prime( \mu + \sigma v_s ) v_s}
                                { \sum_{s=1}^S w_s  e^{ L_i( \mu + \sigma v_s ) } } 
                    \\
        &= -\frac{1}{N} \sum_{i=1}^I 
                            \frac{ e^{L_i^*(\mu,\sigma)} \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) - L_i^*(\mu,\sigma) } L_i^\prime( \mu + \sigma v_s ) v_s}
                                { e^{L_i^*(\mu,\sigma)} \sum_{s=1}^S w_s  e^{ L_i( \mu + \sigma v_s ) - L_i^*(\mu,\sigma) } } 
                    \\
        &= -\frac{1}{N} \sum_{i=1}^I 
                            \frac{ \sum_{s=1}^S w_s e^{ L_i( \mu + \sigma v_s ) - L_i^*(\mu,\sigma) } L_i^\prime( \mu + \sigma v_s ) v_s}
                                { \sum_{s=1}^S w_s  e^{ L_i( \mu + \sigma v_s ) - L_i^*(\mu,\sigma) } } 
                    \\
\end{aligned}
$$

In [322]:
class RCLogit( FittableModel ) : 
    
    type = "Random Coefficient Logit"
    
    def __init__( self , N , I , i , y ) : 
        
        self.Nvars , self.N , self.I , self.i , self.y = 2 , N , I , i , -y
        
        # these matrices "assign" observations to individuals, facilitating 
        # sums over observations within individuals. 
        self.reducer = csr_matrix( (-np.ones(N),(i,np.arange(N))) , shape=(I,N) )
        self.yeducer = csr_matrix( (y,(i,np.arange(N))) , shape=(I,N) ) 
        
        # constraints: sigma is non-negative    
        self.cons = LinearConstraint( np.array([[0.0,1.0]]) , np.array([0.0]) , np.array([np.inf]) )
        
        self.S = 0
    
    def allocate_workspace( self ) : 
        if self.S > 0 :
            self.eU = np.zeros((self.N,self.S),dtype=np.float) # N x S space
            self.ll = np.zeros((self.N,self.S),dtype=np.float) # N x S space
            self.Li = np.zeros((self.I,self.S),dtype=np.float) # I x S space
            self.PL = np.zeros((self.N,self.S),dtype=np.float) # I x S space
            self.LP = np.zeros((self.I,self.S),dtype=np.float) # I x S space
            self.E  = np.zeros((self.I,self.S),dtype=np.float) # I x S space
            self.j  = np.zeros((self.I,3),dtype=np.float) # I x 3 space
            self.LM = np.zeros((self.I,),dtype=np.float) # I space
        return
    
    def basics( self , p ) : 
        theta = p[0] + p[1] * self.v 
        np.exp( np.outer( self.y , theta ) , out=self.eU )
        np.log1p( self.eU , out=self.ll )
        self.Li = self.reducer @ self.ll
        
        # stabilization
        self.LM = np.max( self.Li , axis=1 ) # get largest Li[i,:] -> LM[i]
        self.Li = self.Li - np.tile( self.LM.reshape((self.I,1)) , (1,self.S) ) # subtract max from each Li
        np.exp( self.Li , self.E ) # E[i,:] <- exp{ Li[i,:] - LM[i] }
        self.j[:,0] = self.E @ self.w # each term is at least as large as the minimum weight
        
    def obj( self , p ) :
        self.basics( p ) # this updates things used in both obj and grad
        np.log( self.j[:,0] , out=self.j[:,1] ) # log only the shifted terms
        return - np.sum( self.LM + self.j[:,1] ) / self.N # have to add in the max terms

    def grad( self , p ) :  
        
        self.basics( p ) # this updates things used in both obj and grad
        
        np.divide( self.eU , 1.0 + self.eU , out=self.PL )
        self.LP = self.yeducer @ self.PL # checked, this is doing the intended sub-sums
        np.multiply( self.E , self.LP , out=self.E )
        self.j[:,1] = self.E @ self.w
        
        # if we would guess a loop over i is more efficient, otherwise, loop over s
        if self.I <= self.S : 
            self.E = np.apply_along_axis( lambda e : e * self.v , 1 , self.E )
        else : 
            for s in range(self.S) : self.E[:,s] = self.E[:,s] * self.v[s]
        self.j[:,2] = self.E @ self.w
        
        np.divide( self.j[:,1] , self.j[:,0] , out=self.j[:,1] )
        np.divide( self.j[:,2] , self.j[:,0] , out=self.j[:,2] )
        
        g = np.zeros(2)
        g[0] = - np.sum( self.j[:,1] ) / self.N
        g[1] = - np.sum( self.j[:,2] ) / self.N
        
        return g
    
    def draw_initial_condition( self ) : 
        p0 = randn( self.Nvars )
        p0[1] = np.abs(p0[1])
        return p0
        
    def solve( self , p0=None ) : 
        self.soln = minimize( self.obj , p0 , jac=self.grad , hess=BFGS() , \
                            method='trust-constr' , constraints=self.cons , \
                           options={ 'maxiter' : 100000 } )
        return self.soln
    
    def printx( self ) : 
        if self.soln is None : return "(no solution)"
        return "%0.2f ( %0.2f )" % ( self.soln['x'][0] , self.soln['x'][1] )
    
    def getx( self ) : 
        if self.soln is None : return None
        else : return self.soln['x'][:2]
    
    def kld( self , PT ) : 
        theta = self.soln['x'][0] + self.soln['x'][1] * self.v # mu + sigma v
        PL = np.zeros( self.S )
        ind = np.where( theta > 0.0 )[0]
        PL[ind] = 1.0 / ( 1.0 + np.exp(-theta[ind]) )
        ind = np.where( theta <= 0.0 )[0]
        PL[ind] = np.exp(theta[ind]) ; PL[ind] = PL[ind] / ( 1.0 + PL[ind] )
        P = np.sum( PL * self.w )
        return PT * np.log( PT/P ) + (1.0-PT) * np.log( (1.0-PT)/(1.0-P) )
        

### Sample Average Approximations

The simplest integral approximation comes from just sample averaging: where we draw $S$ samples $v_s$ from a standard gaussian distribution and use weights $w_s = 1/S$ or maybe even $w_s = \phi(v_s)$. This is easy, but is also probably the least efficient approximation we can use. 

In [323]:
class GRCLogitSAA( RCLogit ) : 
        
    type = "Gaussian RC Logit (SAA)"
    
    def __init__( self , N , I , i , y , samples=1000 , weighted=False ) : 
        
        # initialize the superclass
        RCLogit.__init__( self , N , I , i , y )
        
        # if we change parameters from now on, redo
        self.do_update_data = False
        
        # should we PDF-weight the samples? 
        self.weighted = True if weighted else False # "truthy" filter for this param
        
        # get samples
        self.resample( samples=samples )
        
        # allocate workspace
        self.allocate_workspace()
        
        # if we change parameters from now on, redo
        self.do_update_data = True
        
    def resample( self , samples=1000 ) : 
        
        # define basic sampling data: S random normal samples
        self.S = samples
        self.v = randn( self.S )
        
        # "weight" vector for summing/weighting samples
        if self.weighted : 
            self.w = gaussian.pdf( self.v ) ; self.w = self.w / self.w.sum()
        else : 
            self.w = np.ones( self.S ) / self.S
            
        if self.do_update_data :
            self.allocate_workspace()
            
        return
    

### Gauss-Legendre Quadrature 

Divide the real line into $Q+2$ intervals
$$
    (\infty,p_1] \; , \; (p_1,p_2] \; , \; \dotsc \; , \; (p_Q,p_{Q+1}] \; , \; (p_{Q+1},\infty)
$$
using $Q+1$ points 
$$
    p_1 \; < \; p_2 \; < \; \dotsb \; < \; p_{Q+1}
$$
and take 
$$
    \int e^{ L_i( \mu + \sigma v ) }\phi(v)dv
        \;\; \approx \;\; \sum_{q=1}^Q \int_{p_q}^{p_{q+1}} e^{ L_i( \mu + \sigma v ) }\phi(v)dv
$$
presuming $p_{1},p_{Q+1}$ are sufficiently large in magnitude so that
$$
    \int_{-\infty}^{p_{1}} e^{ L_i( \mu + \sigma v ) }\phi(v)dv
        \;\; , \;\; \int_{p_{Q+1}}^{\infty} e^{ L_i( \mu + \sigma v ) }\phi(v)dv
        \;\; \approx \;\; 0. 
$$
This allows us to use standard [Gauss-Legendre](https://en.wikipedia.org/wiki/Gaussian_quadrature#Gauss%E2%80%93Legendre_quadrature) approximations
$$
    \int_{p_q}^{p_{q+1}} e^{ L_i( \mu + \sigma v ) }\phi(v)dv
        \;\; \approx \;\; 
            \triangle_s \sum_{k=1}^K \alpha_k e^{ L_i( \mu + \sigma v_{q,k} ) } \phi(v_{q,k})
           \quad\quad
           v_{q,k} = \triangle_q \xi_k + \Gamma_q
$$
for some $K$-point integration rule with nodes $\xi_k \in [-1,1]$ and weights $\alpha_k$, where
$$
    \triangle_q = \frac{p_{q+1}-p_{q}}{2}
    \quad\quad\text{and}\quad\quad
    \Gamma_q = \frac{p_{q}+p_{q+1}}{2}
$$
Hence
$$
       \int e^{ L_i( \mu + \sigma v ) }\phi(v)dv
           \approx \sum_{q=1}^Q \triangle_q \sum_{k=1}^K \alpha_k e^{ L_i( \mu + \sigma v_{q,k} ) }
                    \phi(v_{q,k})
            = \sum_{s=1}^{QK} w_s e^{ L_i( \mu + \sigma v_s ) } 
$$
letting $w_s \sim \triangle_q \alpha_k \phi(v_s)$ (for an appropriate $s \to (q,k)$ index transformation). In this way we can absorb these quadrature-specific terms into the weights corresponding to specific samples. 

To compute, set
$$
    S = QK
    \quad\quad
    \mathbf{V} = \boldsymbol{\triangle} \boldsymbol{\xi}^\top + \boldsymbol{\Gamma}\mathbf{1}^\top
    \quad\quad
    \mathbf{v} = \mathrm{vec}( \mathbf{V} )
    \quad\quad
    \mathbf{w} = \mathrm{vec}( \; \boldsymbol{\triangle} \boldsymbol{\alpha}^\top * \phi(\mathbf{V}) \; )
$$
and apply the generic weighted sample formulation above. 

In [324]:
class GRCLogitGLQ( RCLogit ) : 
        
    type = "Gaussian RC Logit (GLQ)"
    
    def __init__( self , N , I , i , y , quad_order=3 , partition=None ) : 
    
        # initialize the superclass
        RCLogit.__init__( self , N , I , i , y )
        
        # don't update in the routines below
        self.do_update_data = False
        
        # define basic quadrature data
        self.quadorder( K=quad_order )
        
        # define the basic partition data
        if partition is None : 
            self.partition()
        else : 
            if 'min' not in partition : partition['min'] = -3.0
            if 'max' not in partition : partition['max'] =  4.0
            if 'num' not in partition : partition['num'] =  10
            self.partition( pmin=partition['min'] , pmax=partition['max'] , pnum=partition['num'] )
        
        # define the nodes and weights
        self.define_nodes_and_weights()
        
        # allocate workspace
        self.allocate_workspace()
        
        # if we change parameters from now on, redo
        self.do_update_data = True
        
    def plot_quadrature( self ) :
        plt.figure( )
        for p in self.qp : plt.plot( [p,p] , [0,1] , '--b' )
        plt.plot( self.v , 0.0 * np.ones( self.S ) , '.k' )
        plt.plot( self.v , gaussian.pdf( self.v ) , '-k' )
        return

    def quadorder( self , K=3 ) : 
        
        if K < 1 or K > 5 : 
            raise ArgumentError( "only handling quadrature for K in {1,2,3,4,5}" )
            
        self.K = K
        if K == 1 : 
            self.xi = np.array( [ 0.0 ] )
            self.qw = np.array( [ 2.0 ] )
        elif K == 2 :
            self.xi = np.array( [ -0.57735 , 0.57735 ] )
            self.qw = np.array( [  1.00000 , 1.00000 ] )
        elif K == 3 :
            self.xi = np.array( [ -0.774597 , 0.000000 , 0.774597 ] )
            self.qw = np.array( [  0.555556 , 0.888889 , 0.555556 ] )
        elif K == 4 :
            self.xi = np.array( [ -0.861136 , -0.339981 , 0.339981 , 0.861136 ] )
            self.qw = np.array( [  0.347855 ,  0.652145 , 0.652145 , 0.347855 ] )
        else :
            self.xi = np.array( [ -0.906180 , -0.538469 , 0.000000 , 0.538469 , 0.906180 ] )
            self.qw = np.array( [  0.236927 ,  0.478629 , 0.568889 , 0.478629 , 0.236927 ] )
            
        if self.do_update_data : 
            self.define_nodes_and_weights()
            self.allocate_workspace()
            
        return
        
    def partition( self , pmin=-3.0 , pmax=4.0 , pnum=10 ) : 
        pdel = ( pmax - pmin ) / pnum
        p = np.exp( 0.5 * np.arange( pmin , pmax + 0.5*pdel , pdel ) )
        self.qp = np.concatenate( ( -p[::-1] , [0] , p )  )
        self.Q = len( self.qp ) - 1
        self.delta = ( self.qp[1:] - self.qp[:-1] ) / 2.0 # S-vector, approximation interval radii
        self.gamma = ( self.qp[1:] + self.qp[:-1] ) / 2.0 # S-vector, approximation interval midpoints 
        if self.do_update_data : 
            self.define_nodes_and_weights()
            self.allocate_workspace()
        return
    
    def define_nodes_and_weights( self ) : 
        self.S = self.Q * self.K
        V = np.outer( self.delta , self.xi ) + np.tile( self.gamma.reshape((self.Q,1)) , (1,self.K) )
        self.v = V.copy().flatten()
        self.w = ( np.outer( self.delta , self.qw ) * gaussian.pdf( V ) ).flatten()
        return
    

## idLogit

- - -

The `idLogit` for this situation is
$$
\begin{aligned}
    \max &\quad \frac{1}{N} \sum_{i=1}^I \log( 1 + e^{-y_n(\beta+\delta_{i_n})} )
                    + \frac{\Lambda_1}{N} || \boldsymbol{\delta} ||_1 
                    + \frac{\Lambda_2}{2N} || \boldsymbol{\delta} ||_2^2 \\
    \text{w.r.t.} &\quad \beta , \delta_1, \dotsc , \delta_I \\
    \text{s.to} &\quad \delta_1 + \dotsb + \delta_I = 0
\end{aligned}
$$
or, in smooth NLP form, 
$$
\begin{aligned}
    \max &\quad \frac{1}{N} \sum_{n=1}^N \log( 1 + e^{-y_n(\beta+\delta_{i_n})} )
                    + \frac{\Lambda_1}{N} \sum_{i=1}^I s_i
                    + \frac{\Lambda_2}{2N} || \boldsymbol{\delta} ||_2^2 \\
    \text{w.r.t.} &\quad \beta , \boldsymbol{\delta} , \mathbf{s} \\
    \text{s.to} &\quad \delta_1 + \dotsb + \delta_I = 0 \\
        &\quad \mathbf{s} - \boldsymbol{\delta} \geq \mathbf{0} \;\; , \;\; 
                \mathbf{s} + \boldsymbol{\delta} \geq \mathbf{0}
\end{aligned}
$$
Writing the log likelihood part of the objective as
$$
    \frac{1}{N} \sum_{n=1}^N \log( 1 + e^{\mathbf{z}_n^\top\mathbf{x}} )
    \quad\quad \mathbf{x} = \begin{pmatrix} \beta \\ \boldsymbol{\delta} \end{pmatrix}
$$
the gradient is
$$
    \begin{pmatrix}
        \frac{1}{N} \sum_{n=1}^N P_n(\mathbf{x}) \mathbf{z}_n
        + \begin{pmatrix} 0 \\ \frac{\Lambda_2}{N} \boldsymbol{\delta} \end{pmatrix}
        \\
        \frac{\Lambda_1}{N} \mathbf{1}
        \end{pmatrix}
        \quad\text{where}\quad
        P_n(\mathbf{x}) = \frac{ e^{\mathbf{z}_n^\top\mathbf{x}} }{ 1 + e^{\mathbf{z}_n^\top\mathbf{x}} }
$$
Here the Hessian is even pretty straightforward: The first $1+I$ components are
$$
    \frac{1}{N} \sum_{n=1}^N P_n(\mathbf{x})(1-P_n(\mathbf{x})) \mathbf{z}_n \mathbf{z}_n^\top
        + \frac{\Lambda_2}{N} \begin{pmatrix} 0 & \mathbf{0} \\ \mathbf{0} & \mathbf{I} \end{pmatrix}
$$
and there are no "$\mathbf{s}$" components. Thus Hessian-vector products are
$$
    \mathbf{H}\begin{pmatrix}u\\\mathbf{v}\\\mathbf{w}\end{pmatrix}
        = \begin{pmatrix} 
            \frac{1}{N} \sum_{n=1}^N \left( \mathbf{z}_n^\top
                \begin{pmatrix} u \\ \mathbf{v} \end{pmatrix} \right) P_n(\mathbf{x})(1-P_n(\mathbf{x})) \mathbf{z}_n 
                + \frac{\Lambda_2}{N} \begin{pmatrix} 0 \\ \mathbf{v} \end{pmatrix}
              \\
             \mathbf{0}  
        \end{pmatrix}
$$

In [331]:
class idLogit( FittableModel ) : 
        
    type = "idLogit"
    
    def __init__( self , N , I , i , y , Lambda1=None , Lambda2=None ) : 
        
        self.N , self.I , self.i , self.y = N , I , i , y
        self.Nvars , self.Ncons = 1 + 2*I , 1 + 2*I
        self.Ip1 = I + 1
        
        # map of coefficients to "-y(b+d)" terms
        Znnzs = 2*N
        Zdata = np.zeros( Znnzs )
        Zrows , Zcols = np.zeros( Znnzs , dtype=np.int ) , np.zeros( Znnzs , dtype=np.int )
        
        Zdata[:N] , Zdata[N:] = -y , -y
        Zrows[:N] , Zrows[N:] = np.arange(N) , np.arange(N) 
        Zcols[:N] , Zcols[N:] = 0 , 1 + i
        
        self.Z = csr_matrix( (Zdata,(Zrows,Zcols)) , shape=( N , 1+I ) )
        
        # spy( self.Z )
        
        # constraints
        
        lo , up = np.zeros( self.Ncons ) , np.inf * np.ones( self.Ncons ) ; up[0] = 0.0
        
        Cnnzs = 5*I
        Cdata = np.ones( Cnnzs ) # almost all entries are ones
        Crows , Ccols = np.zeros( Cnnzs , dtype=np.int ) , np.zeros( Cnnzs , dtype=np.int )
        
        Crows[0*I:1*I] , Ccols[0*I:1*I] = 0 , 1 + np.arange(I)
        Crows[1*I:2*I] , Ccols[1*I:2*I] , Cdata[1*I:2*I] = 1 + np.arange(I) , 1 + np.arange(I) , -1.0
        Crows[2*I:3*I] , Ccols[2*I:3*I] = 1 +     np.arange(I) , 1 + I + np.arange(I)
        Crows[3*I:4*I] , Ccols[3*I:4*I] = 1 + I + np.arange(I) , 1 +     np.arange(I)
        Crows[4*I:5*I] , Ccols[4*I:5*I] = 1 + I + np.arange(I) , 1 + I + np.arange(I)
        
        Cmtrx = csr_matrix( (Cdata,(Crows,Ccols)) , shape=( self.Ncons , self.Nvars ) )
        
        # spy( Cmtrx )
            
        self.cons = LinearConstraint( Cmtrx , lo , up )
        
        # initial regularization
        L1 = Lambda1 if Lambda1 is not None else self.N
        L2 = Lambda2 if Lambda2 is not None else self.N
        self.regularize( Lambda1=L1 , Lambda2=L2 )
        
        self.zp = np.zeros((self.N,),dtype=np.float)
        self.eU = np.zeros((self.N,),dtype=np.float)
        self.ll = np.zeros((self.N,),dtype=np.float)
        self.PL = np.zeros((self.N,),dtype=np.float)

    def obj( self , p ) :
        np.exp( self.Z @ p[:self.Ip1] , self.eU )
        np.log1p( self.eU , self.ll )
        return np.sum( self.ll ) / self.N \
                    + self.L1 * np.sum( p[self.Ip1:] ) \
                    + self.L2 * np.sum( p[1:self.Ip1] * p[1:self.Ip1] ) / 2.0
        
    def grad( self , p ) :  
        g = np.zeros( self.Nvars )
        np.exp( self.Z @ p[:self.Ip1] , out=self.eU )
        np.divide( self.eU , 1.0 + self.eU , out=self.PL )
        g[:self.Ip1] = self.Z.T @ self.PL / N
        if( self.L2 > 0.0 ) : g[1:self.Ip1] += self.L2 * p[1:self.Ip1]
        g[self.Ip1:] = self.L1
        return g
    
    def hessp( self , p , v ) :  
        h = np.zeros( self.Nvars )
        zv = self.Z @ v[:self.Ip1]
        np.exp( self.Z @ p[:self.Ip1] , out=self.eU )
        np.divide( self.eU , 1.0 + self.eU , out=self.PL )
        self.PL = self.PL * ( 1.0 - self.PL )
        h[:self.Ip1] = self.Z.T @ ( zv * self.PL ) / N
        if( self.L2 > 0.0 ) : h[1:self.Ip1] += self.L2 * v[1:self.Ip1]
        return h
    
    def regularize( self , Lambda1=None , Lambda2=None ) : 
        if Lambda1 is not None : self.L1 = Lambda1 / self.N
        if Lambda2 is not None : self.L2 = Lambda2 / self.N
            
    def draw_initial_condition( self ) : 
        p0 = randn( self.Nvars )
        p0[1:] = p0[1:] - p0[1:].mean()
        return p0
    
    def solve( self , p0=None ) : 
        self.soln = minimize( self.obj , p0 , jac=self.grad , hessp=self.hessp , \
                            method='trust-constr' , constraints=self.cons , \
                           options={ 'maxiter' : 100000 } )
        return self.soln
    
    def trace( self , p0=None , Lambda1s=None , alpha=None ) : 
        p0 = self.draw_initial_condition() if p0 is None else p0.flatten()
        if Lambda1s is None : Lambda1s = 10.0 ** np.arange(1,10,1)[::-1]
        if alpha is None : alpha = 1.0
        self.trace = [ None for l in range(Lambda1s.size) ]
        for l in range(Lambda1s.size) : 
            L = Lambda1s[l]
            self.regularize( Lambda1=L/self.N , Lambda2=alpha*L/self.N )
            self.trace[l] = self.fit( p0=p0 )
            p0 = trace[l].x
        return self
    
    def printx( self ) : 
        if self.soln is None : return "(no solution)"
        k2 , p = normaltest( self.soln['x'][1:I+1] )
        return "%0.2f ( %0.2f , %0.4f )" % ( self.soln['x'][0] , self.soln['x'][1:I+1].std() , p )
    
    def getx( self ) : 
        if self.soln is None : return None
        else : 
            k2 , p = normaltest( self.soln['x'][1:I+1] )
            return np.array( [ self.soln['x'][0] , self.soln['x'][1:I+1].std() , p ] )
    
    def kld( self , PT ) : 
        _ , Ni = np.unique( i , return_counts=True )
        Wi = Ni / self.N # individual observation weights
        B = self.soln['x'][1:I+1] + self.soln['x'][0]
        PL = np.zeros( self.I )
        ind = np.where( B >  0.0 )[0] ; PL[ind] = 1.0 / ( 1.0 + np.exp(-B[ind]) )
        ind = np.where( B <= 0.0 )[0] ; PL[ind] = np.exp(B[ind]) ; PL[ind] = PL[ind] / ( 1.0 + PL[ind] )
        P = np.sum( PL * Wi )
        return PT * np.log( PT/P ) + (1.0-PT) * np.log( (1.0-PT)/(1.0-P) )
        

# Data Generating Processes

- - -

Let's define some simple data generating processes that map $(N,I,\mathbf{i})$ (along with possibly other parameters) to draws of $\mathbf{y}$. Below we define functions that will randomly draw from a Logit, a Latent Class Logit, and a (Gaussian) Random Coefficients Logit model. 

In [377]:
# multinomial Logit data generating process
def mnl_dgp( N , I , i ) : 
    
    pT = 2.0 * rand() - 1.0
    
    print( pT )
    
    if pT > 0 : PT = 1.0 / ( 1.0 + np.exp( -pT ) )
    else : PT = np.exp( pT ) ; PT = PT / ( 1.0 + PT )
    y = 2.0 * ( rand(N) <= PT ) - 1.0
    
    return y , PT

# Latent Class Logit data generating process
def lcl_dgp( N , I , i , C ) : 
    
    w = rand(C) ; w = w / w.sum()
    pT = 2.0 * rand(C) - 1.0
    ic = randc(C,I,w)
    eU = np.exp( pT[ ic[i] ] )
    PL = eU / ( 1.0 + eU )
    y = 2.0 * ( rand(N) <= PL ) - 1.0
    
    PL = np.exp( pT ) ; PL = PL / ( 1.0 + PL )
    PT = np.sum( PL * w )
    
    return y , PT

# Gaussian Random Coefficient Logit data generating process
def grc_dgp( N , I , i ) : 
    
    pT = rand(2) ; pT[0] = 2.0 * pT[0] - 1.0
    
    print( pT )
    
    v  = pT[0] + pT[1] * randn(I)
    eU = np.exp( v[i] )
    PL = eU / ( 1.0 + eU )
    y  = 2.0 * ( rand(N) <= PL ) - 1.0
    
    v   = pT[0] + pT[1] * randn( int(1e6) )
    PL  = np.zeros( int(1e6) )
    ind = np.where( v >  0.0 )[0] ; PL[ind] = 1.0 / ( 1.0 + np.exp( -v[ind] ) )
    ind = np.where( v <= 0.0 )[0] ; PL[ind] = np.exp( v[ind] ) ; PL[ind] = PL[ind] / ( 1.0 + PL[ind] )
    PT  = PL.mean()
    
    return y , PT

# Simulation Experiments

- - -

To draw a simulation experiment, we need to choose or draw $(N,I)$, then $i_n \in \{1,\dotsc,I\}$ for all $n$, and then choice dummies $y_n$ following some data generating process. Below we wrap the second two parts in a single function, `draw_experiment`. Then we create model object instances from the resultant data for each of our model types. 

Just to make sure everything works, we run a test that just (a) checks the gradient for each model and (b) fits the model on the data. 

In [379]:
def draw_experiment( N , I , dgp ) : 
    i = randi( I , N ) ; y , PT = dgp( N , I , i )
    return i , y , PT

N , I = 1000 , 100
i , y , PT = draw_experiment( N , I , mnl_dgp )

# define simple Logit
mnl = Logit( N , y )

# define latent class Logit instances, for a specific number of modeled classes
M = 3
lcl = LatentClassLogit( N , I , i , y , M )
ccl = ClassClassLogit( N , I , i , y , M )

# define sample average and gauss-legendre random coefficient Logit instances
saa = GRCLogitSAA( N , I , i , y )
glq = GRCLogitGLQ( N , I , i , y )

# finally, define an idLogit instance
idl = idLogit( N , I , i , y )

def basic_test( prob ) : 
    print( "%s :: Checking gradient..." % ( prob.type ) )
    prob.grad_check(  )
    print( "%s :: Fitting model..." % ( prob.type ) )
    soln = prob.fit().soln
    print( "%s :: Solver ran in %0.6f seconds" % ( prob.type , soln.solvertime ) )
    print( "%s :: Solver message: %s" % ( prob.type , soln.message ) )
    print( "%s :: Formatted solution: %s" % ( prob.type , prob.printx() ) )
    kld = prob.kld( PT )
    print( "%s :: Kullback-Leibler Divergence at solution : %0.6f" % ( prob.type , kld ) )
    print( "%s :: Relative Average Likelihood at solution : %0.3f" % ( prob.type , np.exp( - kld ) ) )
    
basic_test( mnl ) ; print(" ")
basic_test( lcl ) ; print(" ")
basic_test( ccl ) ; print(" ")
basic_test( saa ) ; print(" ")
basic_test( glq ) ; print(" ")
basic_test( idl ) ; print(" ")

-0.24907225351595952
Logit :: Checking gradient...
0.1000000000000000 , 0.0124346836740238
0.0100000000000000 , 0.0012460218368490
0.0010000000000000 , 0.0001246231591542
0.0001000000000000 , 0.0000124625235973
0.0000100000000000 , 0.0000012462567930
0.0000010000000000 , 0.0000001246428802
0.0000001000000000 , 0.0000000132875108
0.0000000100000000 , 0.0000000011453885
0.0000000010000000 , 0.0000001010654607
0.0000000001000000 , 0.0000016752913787
Logit :: Fitting model...
Logit :: Solver ran in 0.012133 seconds
Logit :: Solver message: `gtol` termination condition is satisfied.
Logit :: Formatted solution: -0.28
Logit :: Kullback-Leibler Divergence at solution : 0.000101
Logit :: Relative Average Likelihood at solution : 1.000
 
Latent Class Logit :: Checking gradient...
0.1000000000000000 , 0.0074115349231856
0.0100000000000000 , 0.0006851778380661
0.0010000000000000 , 0.0000678640654999
0.0001000000000000 , 0.0000067797812024
0.0000100000000000 , 0.0000006779099719
0.0000010000000000

Ok, now that we have some confidence that everything actually works, we can run some real tests. 

The following code helps us abstract away from the mechanics behind doing the tests themselves. 

In [345]:

def create_model_object( code , N , I , i , y , data={} ) : 
    """
    This is a convenience function for instantiating model objects from a 
    code, basic data shared by all models, and optional additional data that
    a class might require or use. 
    """
    if data is None : data = {} # generic empty object for use 
    if   code == "mnl" : return Logit( N , y[:N] )
    elif code == "lcl" : return LatentClassLogit( N , I , i[:N] , y[:N] , **data )
    elif code == "ccl" : return ClassClassLogit( N , I , i[:N] , y[:N] , **data )
    elif code == "saa" : return GRCLogitSAA( N , I , i[:N] , y[:N] )
    elif code == "glq" : return GRCLogitGLQ( N , I , i[:N] , y[:N] )
    elif code == "idl" : return idLogit( N , I , i[:N] , y[:N] , **data )
    else : return None
    
def fit_model( prob , p0=None , PT=None , timeout=None , verbose=True ) : 
    
    res = { 'N' : prob.N , 'p' : None , 'x' : None , 'k' : None , \
               't' : None , 'e' : True , 'm' : None , 's' : None }
    
    try : 
        prob.fit( maxtime=timeout , p0=p0 )
        if prob.soln is None    : raise Exception( "indeterminate solver error" )
        if prob.soln['timeout'] : raise Exception( "solver timeout" )
        if prob.soln['error']   : raise prob.soln['message']
        res['s'] = prob.soln['status']
        res['p'] = prob.printx()
        res['x'] = prob.getx()
        res['t'] = prob.soln['solvertime']
        res['e'] = prob.soln['error']
        res['m'] = prob.soln['message']
        if ( PT is not None ) and ( not prob.soln['error'] ) : res['k'] = prob.kld( PT )
        if verbose : 
            print( "%s :: Solve attempt finished (%i) for N = %i" \
                      % ( prob.type , prob.soln['status'] , prob.N ) )
            
    except Exception as e : 
        if verbose : print( "%s :: solve exception occurred: %s" % ( prob.type , e ) )
        try : solvertime = prob.soln['solvertime']
        except AttributeError as ae : solvertime = None
        res['t'] , res['e'] , res['m'] = solvertime , True , e
        
    return res
    
def fit_sequence( Ns , I , i , y , code , data , PT=None , timeout=None , chain=False , verbose=True ) :
    """
    This is a convenience function for running a sequence of fits over 
    increasing numbers of observations with structured initial conditions. 
    We can either "chain" the solves by passing the solution for one problem
    to the next, or not and keep the same inital
    """
    results , prob_type , p0 = [] , None , None
    for N in Ns : 
        prob , res = None , None
        try :
            prob = create_model_object( code , N , I , i , y , data )
            if prob_type is None : prob_type = prob.type
        except Exception as e : 
            print( "Error creating model object: " , e )
        if prob is not None : 
            res = fit_model( prob , p0=p0 , PT=PT , timeout=timeout , verbose=verbose )
            p0 = prob.soln['x'] if chain else prob.soln['x0']
        results.append( res )
    return results, prob_type

class FitSequenceTrial( object ) :
    """
    This is a wrapper class to enable parallelization with pool.map by 
    making a callable object instance storing the desired model data. 
    """
    def __init__( self , Ns , I , i , y , PT=None , timeout=None , verbose=None ) : 
        self.Ns , self.I , self.i , self.y = Ns , I , i , y
        self.timeout , self.verbose = timeout , verbose
    def __call__( self , params ) : # expects params = { code , data , trial }  
        return fit_sequence( self.Ns , self.I , self.i , self.y , params['code'] , params['data'] , \
                                PT=PT , timeout=self.timeout , verbose=self.verbose )
    

Now we define a class `ModelFitExp` to facilitate running experiments over the sample size ($N$) -- with the same observsational data -- for a variety of models. 

In [371]:

class ModelFitExp( object ) : 
    
    def __init__( self , I=0 , dgp=None , obsvnums=None , timeout=None , \
                     models=None , trials=None , processes=None , verbose=True ) : 
        self.I = I 
        self.dgp = dgp
        self.timeout = timeout
        self.models = {}
        self.verbose = verbose
        self.T = trials if trials is not None else 1 # one trial by default
        if models is not None : 
            for m in models : 
                self.add_model( m['code'] , data=( m['data'] if 'data' in m else None ) )
        self.results = {}
        self.Ns = []
        if obsvnums is None : 
            self.Ns = [ int(N) for N in 10.0 ** np.arange( 3 , 6.1 , 0.1 ) ]
        else : 
            self.Ns = obsvnums
        self.P = 1
        self.parallelize( processes )
        self.iy_saved = False
    
    def be_verbose( self ) : 
        self.verbose = True
        
    def be_quiet( self ) : 
        self.verbose = False
    
    def set_obsnums( self , Nmin , Nmax , Nnum ) : 
        try : 
            Ndel = ( Nmax - Nmin ) / Nnum
            self.Ns = [ int(N) for N in 10.0 ** np.arange( Nmin , Nmax + 0.5*Ndel , Ndel ) ]
        except Exception as e : 
            self.Ns = None
            raise e
            
    def set_dgp( self , dgp ) : 
        self.dgp = dgp
    
    def set_timeout( self , timeout ) : 
        self.timeout = timeout
        
    def add_model( self , code , data=None ) : 
        mid = random_string( 16 )
        self.models[mid] = { 'code' : code , 'data' : data }
        return mid
        
    def del_model( self , mid ) : 
        if mid in self.models : 
            del self.models[mid]
            
    def clear_models( self ) : 
        del self.models
        self.models = {}
        
    def reset_results( self ) : 
        del self.results
        self.results = {}
        
    def parallelize( self , procs ) : 
        try : self.P = int( procs )
        except Exception as e : pass
        if self.P < 1 : self.P = 1
        
    def print_models( self ) : 
        for k in self.models : 
            print( "(%s) %s %s" % ( k , self.models[k]['code'] , \
                                   str(self.models[k]['data']) if self.models[k]['data'] is not None else "" ) )
        
    def print_results( self ) : 
        for k in self.results : 
            print( "%s%s" % ( self.models[k]['type'] , "" if self.models[k]['data'] is None \
                                 else ", %s" % str(self.models[k]['data']) ) )
            t = 1
            for trial in self.results[k] : 
                for i in trial : 
                    if i['e'] : 
                        print( "%i , %i (%s) , %s" \
                                  % ( t , i['N'] , "-" if i['s'] is None else "%i" % i['s'] , i['m'] ) )
                    else :
                        print( "%i , %i (%s) , %0.2fs , %0.3f , %s " \
                                  % ( t , i['N'] , "-" if i['s'] is None else "%i" % i['s'] , \
                                       i['t'] , np.exp( - i['k'] ) , i['p'] ) )
                print( " " )
                t += 1
    
    def run( self , trials=None , save=False , resample=False ) : 
        
        if trials is not None : self.T = trials
        if self.iy_saved and ( not resample ) : 
            i , y = self.i , self.y
        else : 
            i , y , self.PT = draw_experiment( self.Ns[-1] , self.I , self.dgp )
            if save : 
                self.i , self.y , self.iy_saved = i , y , True
                
        # parallelize? May not be worthwhile. 
        if self.P > 1 : 
            map_params = []
            for k in self.models : 
                if k not in self.results : self.results[k] = [ [] for t in range(self.T) ]
                for t in range( self.T ) : 
                    map_params.append( { 
                        'code'  : self.models[k]['code'] ,
                        'data'  : self.models[k]['data'] ,
                        'trial' : t
                    } )
            with ThreadPool( processes=self.P ) as pool : 
                FST = FitSequenceTrial( self.Ns , self.I , i , y , \
                                           PT=self.PT , timeout=self.timeout , verbose=self.verbose )
                results = pool.map( FST , map_params )
            j = 0
            for k in self.models : 
                for t in range( self.T ) : 
                    self.results[k][t] , self.models[k]['type'] = results[j][0] , results[j][1]
                    j += 1
                    
        # serial trials; seems to be the best
        else : 
            for k in self.models : 
                if k not in self.results : self.results[k] = []
                for t in range( self.T ) : 
                    while t >= len( self.results[k] ) : self.results[k].append( [] )
                    self.results[k][t] , self.models[k]['type'] \
                        = fit_sequence( self.Ns , self.I , i , y , \
                                        self.models[k]['code'] , self.models[k]['data'] , \
                                        PT=self.PT , timeout=self.timeout , verbose=self.verbose )
            

### Logit data

Let's take the simplest case in the data, the Logit DGP, but try out each model with more and more data. This will help us _start_ to gauge comparative efficiency. To start, we will apply a one minute (60 second) timeout on the fit attempts. 

In [378]:

mdls = [
    { 'code' : 'mnl' } , 
    # { 'code' : 'lcl' , 'data' : { 'C' : 3 } } , 
    # { 'code' : 'ccl' , 'data' : { 'C' : 3 } } , 
    # { 'code' : 'saa' } , 
    { 'code' : 'glq' } , 
    { 'code' : 'idl' , 'data' : { 'Lambda1' : 10.0 , 'Lambda2' : 1.0 } } , 
    { 'code' : 'idl' , 'data' : { 'Lambda1' :  1.0 , 'Lambda2' : 1.0 } } , 
    { 'code' : 'idl' , 'data' : { 'Lambda1' :  0.1 , 'Lambda2' : 1.0 } } , 
]
fitr = ModelFitExp( I=100 , dgp=mnl_dgp , timeout=60 , models=mdls , trials=2 , verbose=False )
# fitr.set_obsnums( 3 , 6 , 25 )
# fitr.set_obsnums( 3 , 5 , 20 )
fitr.set_obsnums( 3 , 4 , 10 )

fitr.run()

fitr.print_results()


0.4884605886768756
Logit
1 , 1000 (1) , 0.03s , 0.998 , 0.37 
1 , 1258 (1) , 0.04s , 0.999 , 0.38 
1 , 1584 (1) , 0.04s , 0.999 , 0.40 
1 , 1995 (1) , 0.03s , 1.000 , 0.45 
1 , 2511 (1) , 0.03s , 1.000 , 0.44 
1 , 3162 (1) , 0.03s , 1.000 , 0.46 
1 , 3981 (1) , 0.03s , 1.000 , 0.50 
1 , 5011 (1) , 0.03s , 1.000 , 0.51 
1 , 6309 (1) , 0.03s , 1.000 , 0.49 
1 , 7943 (1) , 0.03s , 1.000 , 0.50 
1 , 10000 (1) , 0.03s , 1.000 , 0.50 
 
2 , 1000 (1) , 0.03s , 0.998 , 0.37 
2 , 1258 (1) , 0.03s , 0.999 , 0.38 
2 , 1584 (1) , 0.03s , 0.999 , 0.40 
2 , 1995 (1) , 0.03s , 1.000 , 0.45 
2 , 2511 (1) , 0.03s , 1.000 , 0.44 
2 , 3162 (1) , 0.03s , 1.000 , 0.46 
2 , 3981 (1) , 0.03s , 1.000 , 0.50 
2 , 5011 (1) , 0.03s , 1.000 , 0.51 
2 , 6309 (1) , 0.03s , 1.000 , 0.49 
2 , 7943 (1) , 0.03s , 1.000 , 0.50 
2 , 10000 (1) , 0.03s , 1.000 , 0.50 
 
Gaussian RC Logit (GLQ)
1 , 1000 (1) , 0.09s , 0.998 , 0.37 ( 0.18 ) 
1 , 1258 (1) , 0.10s , 0.999 , 0.38 ( 0.09 ) 
1 , 1584 (1) , 0.11s , 0.999 , 0.40 ( 0

### Random (Gaussian) Coefficient Logit data

Let's look at the Gaussian coefficient case now. 

In [372]:

mdls = [
    { 'code' : 'mnl' } , 
    { 'code' : 'saa' } , 
    { 'code' : 'glq' } , 
    { 'code' : 'idl' , 'data' : { 'Lambda1' : 10.0 , 'Lambda2' : 0.0 } } , 
    { 'code' : 'idl' , 'data' : { 'Lambda1' :  1.0 , 'Lambda2' : 0.0 } } , 
    { 'code' : 'idl' , 'data' : { 'Lambda1' :  0.1 , 'Lambda2' : 0.0 } } , 
]
fitr = ModelFitExp( I=100 , dgp=grc_dgp , timeout=60 , models=mdls , trials=1 , verbose=False )
# fitr.set_obsnums( 3 , 6 , 25 )
fitr.set_obsnums( 3 , 5 , 15 )

fitr.run()
fitr.print_results()


[0.33182725 0.38897958]
Logit
1 , 1000 (1) , 0.04s , 0.999 , 0.39 
1 , 1359 (1) , 0.03s , 1.000 , 0.34 
1 , 1847 (1) , 0.03s , 1.000 , 0.34 
1 , 2511 (1) , 0.03s , 1.000 , 0.35 
1 , 3414 (1) , 0.03s , 1.000 , 0.35 
1 , 4641 (1) , 0.03s , 1.000 , 0.37 
1 , 6309 (1) , 0.03s , 1.000 , 0.36 
1 , 8576 (1) , 0.03s , 1.000 , 0.38 
1 , 11659 (1) , 0.03s , 1.000 , 0.37 
1 , 15848 (1) , 0.04s , 1.000 , 0.37 
1 , 21544 (1) , 0.03s , 1.000 , 0.36 
1 , 29286 (1) , 0.03s , 1.000 , 0.36 
1 , 39810 (1) , 0.03s , 1.000 , 0.35 
1 , 54116 (1) , 0.03s , 1.000 , 0.35 
1 , 73564 (1) , 0.04s , 1.000 , 0.35 
1 , 100000 (1) , 0.04s , 1.000 , 0.35 
 
Gaussian RC Logit (SAA)
1 , 1000 (1) , 0.45s , 0.999 , 0.43 ( 0.55 ) 
1 , 1359 (1) , 0.70s , 1.000 , 0.34 ( 0.44 ) 
1 , 1847 (1) , 0.85s , 1.000 , 0.35 ( 0.37 ) 
1 , 2511 (1) , 1.11s , 1.000 , 0.35 ( 0.33 ) 
1 , 3414 (1) , 1.55s , 1.000 , 0.35 ( 0.35 ) 
1 , 4641 (1) , 2.71s , 1.000 , 0.37 ( 0.36 ) 
1 , 6309 (1) , 3.74s , 1.000 , 0.37 ( 0.40 ) 
1 , 8576 (1) , 25.82s